In [ ]:
import dataset_tools as dtools
import os 
import json
import random
import os 
import shutil

In [ ]:
dtools.download(dataset='Cocoa Diseases', dst_dir='~/dataset-ninja/')

In [ ]:
annot = "GoodLabels"
datasetPath = "GoodImages"
def jsonToTxt(pathToAnnots):
    os.makedirs(annot, exist_ok=True)
    for n in os.listdir(pathToAnnots):
        if not n.endswith('.json'):
            continue
        with open(os.path.join(pathToAnnots, n)) as json_file:
            with open(os.path.join(annot, n[:-5] + ".txt"), "a") as txt_file:
                print(n)
                data = json.load(json_file)
                for nr in data["objects"]:
                    idNum = nr["classId"]
                    points = nr["points"]["exterior"]
                    # write txt file
                    txt_file.write(f"{idNum} {points[0][0]} {points[0][1]} {points[1][0]} {points[1][1]}\n")
jsonToTxt("dataset-ninja/cocoa-diseases/all/ann/")

In [ ]:

def train_test_split(path,labelPath,outputPath,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")

    #exculde files without jpg extension

    files = list(set([name[:-4] for name in os.listdir(path) if name.endswith('.jpg')])) ## removing duplicate names i.e. counting only number of images

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories
    train_path_img = outputPath + "images/train/"
    train_path_label = outputPath + "labels/train/"
    val_path_img = outputPath + "images/val/"
    val_path_label = outputPath + "labels/val/"


    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)



    ### ----------- copying images to train folder
    for filex in files[:train_size]:
      print(filex)
      if filex == 'classes':
          continue
          
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(labelPath+"/" + filex + '.jpg'+'.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in neg_images:
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in files[train_size:]:
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(labelPath+"/" + filex + '.jpg'+'.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

train_test_split("dataset-ninja/cocoa-diseases/all/img/",annot,datasetPath)

